## Data source
https://www.kaggle.com/datasets/whenamancodes/student-performance

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
port_df=pd.read_csv('student-por.csv',sep=';')

In [3]:
port_df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


We can treat both dataframes as separate and solve the feature relevance and prediction problem for both. Comparing the results can give us good insights on any differences/similarities between math and language scores.

## Portugese grades - feature importance
There are 3 grades given in the dataset. We will create an average overall grade as the target variable.

In [4]:
port_df['Grade'] =port_df[['G1','G2','G3']].mean(axis = 1)

In [5]:
port_df = port_df.drop(['G1', 'G2', 'G3'], axis=1)

In [6]:
port_df.isna().sum()

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
Grade         0
dtype: int64

No missing information

In [7]:
port_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   school      649 non-null    object 
 1   sex         649 non-null    object 
 2   age         649 non-null    int64  
 3   address     649 non-null    object 
 4   famsize     649 non-null    object 
 5   Pstatus     649 non-null    object 
 6   Medu        649 non-null    int64  
 7   Fedu        649 non-null    int64  
 8   Mjob        649 non-null    object 
 9   Fjob        649 non-null    object 
 10  reason      649 non-null    object 
 11  guardian    649 non-null    object 
 12  traveltime  649 non-null    int64  
 13  studytime   649 non-null    int64  
 14  failures    649 non-null    int64  
 15  schoolsup   649 non-null    object 
 16  famsup      649 non-null    object 
 17  paid        649 non-null    object 
 18  activities  649 non-null    object 
 19  nursery     649 non-null    o

In [8]:
#seperate categorical columns and numerical columns
cat_cols = port_df.select_dtypes(include=['object'])
cat_cols

,school,sex,address,famsize,Pstatus,Mjob,Fjob,reason,guardian,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic
0,GP,F,U,GT3,A,at_home,teacher,course,mother,yes,no,no,no,yes,yes,no,no
1,GP,F,U,GT3,T,at_home,other,course,father,no,yes,no,no,no,yes,yes,no
2,GP,F,U,LE3,T,at_home,other,other,mother,yes,no,no,no,yes,yes,yes,no
3,GP,F,U,GT3,T,health,services,home,mother,no,yes,no,yes,yes,yes,yes,yes
4,GP,F,U,GT3,T,other,other,home,father,no,yes,no,no,yes,yes,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,R,GT3,T,services,other,course,mother,no,no,no,yes,no,yes,yes,no
645,MS,F,U,LE3,T,teacher,services,course,mother,no,yes,no,no,yes,yes,yes,no
646,MS,F,U,GT3,T,other,other,course,mother,no,no,no,yes,yes,yes,no,no
647,MS,M,U,LE3,T,services,services,course,mother,no,no,no,no,no,yes,yes,no


In [9]:
cat_cols['Mjob'].value_counts()

other       258
services    136
at_home     135
teacher      72
health       48
Name: Mjob, dtype: int64

In [10]:
one_hot_encoded_data = pd.get_dummies(cat_cols)
one_hot_encoded_data

,school_GP,school_MS,sex_F,sex_M,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,Pstatus_T,...,activities_no,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes
0,1,0,1,0,0,1,1,0,1,0,...,1,0,0,1,0,1,1,0,1,0
1,1,0,1,0,0,1,1,0,0,1,...,1,0,1,0,0,1,0,1,1,0
2,1,0,1,0,0,1,0,1,0,1,...,1,0,0,1,0,1,0,1,1,0
3,1,0,1,0,0,1,1,0,0,1,...,0,1,0,1,0,1,0,1,0,1
4,1,0,1,0,0,1,1,0,0,1,...,1,0,0,1,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,0,1,1,0,1,0,1,0,0,1,...,0,1,1,0,0,1,0,1,1,0
645,0,1,1,0,0,1,0,1,0,1,...,1,0,0,1,0,1,0,1,1,0
646,0,1,1,0,0,1,1,0,0,1,...,0,1,0,1,0,1,1,0,1,0
647,0,1,0,1,0,1,0,1,0,1,...,1,0,1,0,0,1,0,1,1,0


In [11]:
num_cols = port_df.select_dtypes(include = ['int64'])
num_cols

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences
0,18,4,4,2,2,0,4,3,4,1,1,3,4
1,17,1,1,1,2,0,5,3,3,1,1,3,2
2,15,1,1,1,2,0,4,3,2,2,3,3,6
3,15,4,2,1,3,0,3,2,2,1,1,5,0
4,16,3,3,1,2,0,4,3,2,1,2,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,19,2,3,1,3,1,5,4,2,1,2,5,4
645,18,3,1,1,2,0,4,3,4,1,1,1,4
646,18,1,1,2,2,0,1,1,1,1,1,5,6
647,17,3,1,2,1,0,2,4,5,3,4,2,6


In [12]:
cols1=num_cols.columns
cols1

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'],
      dtype='object')

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler().fit(num_cols[cols1])
num_cols[cols1]=scaler.transform(num_cols[cols1])

In [14]:
num_cols

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences
0,0.428571,1.00,1.00,0.333333,0.333333,0.000000,0.75,0.50,0.75,0.00,0.00,0.50,0.1250
1,0.285714,0.25,0.25,0.000000,0.333333,0.000000,1.00,0.50,0.50,0.00,0.00,0.50,0.0625
2,0.000000,0.25,0.25,0.000000,0.333333,0.000000,0.75,0.50,0.25,0.25,0.50,0.50,0.1875
3,0.000000,1.00,0.50,0.000000,0.666667,0.000000,0.50,0.25,0.25,0.00,0.00,1.00,0.0000
4,0.142857,0.75,0.75,0.000000,0.333333,0.000000,0.75,0.50,0.25,0.00,0.25,1.00,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,0.571429,0.50,0.75,0.000000,0.666667,0.333333,1.00,0.75,0.25,0.00,0.25,1.00,0.1250
645,0.428571,0.75,0.25,0.000000,0.333333,0.000000,0.75,0.50,0.75,0.00,0.00,0.00,0.1250
646,0.428571,0.25,0.25,0.333333,0.333333,0.000000,0.00,0.00,0.00,0.00,0.00,1.00,0.1875
647,0.285714,0.75,0.25,0.333333,0.000000,0.000000,0.25,0.75,1.00,0.50,0.75,0.25,0.1875


In [15]:
target = port_df['Grade']

In [16]:
final_df = pd.concat([num_cols,one_hot_encoded_data,target], axis=1)

In [17]:
final_df.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,activities_yes,nursery_no,nursery_yes,higher_no,higher_yes,internet_no,internet_yes,romantic_no,romantic_yes,Grade
0,0.428571,1.00,1.00,0.333333,0.333333,0.0,0.75,0.50,0.75,0.00,...,0,0,1,0,1,1,0,1,0,7.333333
1,0.285714,0.25,0.25,0.000000,0.333333,0.0,1.00,0.50,0.50,0.00,...,0,1,0,0,1,0,1,1,0,10.333333
2,0.000000,0.25,0.25,0.000000,0.333333,0.0,0.75,0.50,0.25,0.25,...,0,0,1,0,1,0,1,1,0,12.333333
3,0.000000,1.00,0.50,0.000000,0.666667,0.0,0.50,0.25,0.25,0.00,...,1,0,1,0,1,0,1,0,1,14.000000
4,0.142857,0.75,0.75,0.000000,0.333333,0.0,0.75,0.50,0.25,0.00,...,0,0,1,0,1,1,0,1,0,12.333333


In [18]:
to_drop=pd.DataFrame(final_df.corr()['Grade'][final_df.corr()['Grade']>=0.01])



In [19]:
to_drop

,Grade
Medu,0.265905
Fedu,0.227918
studytime,0.261630
famrel,0.070550
school_GP,0.295035
sex_F,0.118333
address_U,0.167455
famsize_LE3,0.045694
Pstatus_T,0.011045
Mjob_health,0.107438


In [20]:
data = final_df[list(to_drop.index)]
data.head()

,Medu,Fedu,studytime,famrel,school_GP,sex_F,address_U,famsize_LE3,Pstatus_T,Mjob_health,...,guardian_father,schoolsup_no,famsup_yes,paid_no,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_no,Grade
0,1.00,1.00,0.333333,0.75,1,1,1,0,0,0,...,0,0,0,1,0,1,1,0,1,7.333333
1,0.25,0.25,0.333333,1.00,1,1,1,0,1,0,...,1,1,1,1,0,0,1,1,1,10.333333
2,0.25,0.25,0.333333,0.75,1,1,1,1,1,0,...,0,0,0,1,0,1,1,1,1,12.333333
3,1.00,0.50,0.666667,0.50,1,1,1,0,1,1,...,0,1,1,1,1,1,1,1,0,14.000000
4,0.75,0.75,0.333333,0.75,1,1,1,0,1,0,...,1,1,1,1,0,1,1,0,1,12.333333


Some of the features have little to no correlation with grades. Anything below 0.01 can be dropped.

In [33]:
data.to_csv('Math_preprocessed.csv')